In [3]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
from keras.models import Model
from itertools import groupby
import os
import pandas as pd

In [3]:
model = ResNet50(weights='imagenet')

102858752/102853048 [==============================] - 567s 6us/step


In [6]:
vecModel = Model(inputs=model.input, outputs=model.get_layer('avg_pool').output)

In [102]:
def getVectors(info,videoName):
    img = image.load_img('trainingImages/'+videoName+'/'+info[0], target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return vecModel.predict(x)[0]

token2Num = {'Burpees':1,'JumpingJacks':2,'MountainClimber':3,'Plank':4,'PushUps':5,'Squat':6,'Other':0}
num2Token = {1:'Burpees',2:'JumpingJacks',3:'MountainClimber',4:'Plank',5:'PushUps',6:'Squat',0:'Other'}
def getNum(x): return token2Num.get(x, 0)
def getToken(x): return num2Token.get(x, 'Other')

In [108]:
# session_id, label, seconds, i
def generateEmbeddingsForVideo(videoName):
    files = list(map(lambda x:[x,x.split('_')],list(filter(lambda x:'.png' in x,os.listdir('trainingImages/'+videoName+'/')))))
    files.sort(key=lambda x:x[1][0])

    out = []
    for key, group in groupby(files, lambda x: x[1][0]):
        group = list(group)
        group.sort(key=lambda x:x[1][2])
        if(len(group)>=5):
            vecs=[]
            for info in group:
                vecs.append(getVectors(info,videoName))
            for i in range(5,len(vecs)+1):
                out.append([getNum(info[1][1]), np.concatenate(vecs[i-5:i])])
    df = pd.DataFrame(out,columns =['label', 'features'])
    df.to_pickle('trainingEmbeddings/'+videoName)
    print('Saved embeddings for video '+videoName)

for video in ['DIY015','DIY021','DIY08']:
    generateEmbeddingsForVideo(video)

In [4]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [38]:
df = pd.concat(list(map(lambda x:pd.read_pickle('trainingEmbeddings/'+x),['DIY015','DIY021','DIY08'])))


In [40]:
dataset = df.values
X = np.stack(dataset[:,1], axis=0)
Y = dataset[:,0]
dummy_y = np_utils.to_categorical(Y)

Baseline: 100.00% (0.00%)


In [41]:
model = Sequential()
model.add(Dense(100, input_dim=10240, activation='relu'))
model.add(Dense(7, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [42]:
model.fit(X, dummy_y, epochs=150, batch_size=10)

Epoch 1/150
447/447 [==============================] - 1s 3ms/step - loss: 0.4823 - acc: 0.8770
Epoch 2/150
447/447 [==============================] - 0s 706us/step - loss: 0.0509 - acc: 0.9799
Epoch 3/150
447/447 [==============================] - 0s 718us/step - loss: 0.0062 - acc: 0.9978
Epoch 4/150
447/447 [==============================] - 0s 757us/step - loss: 0.0033 - acc: 1.0000
Epoch 5/150
447/447 [==============================] - 0s 696us/step - loss: 9.7057e-04 - acc: 1.0000
Epoch 6/150
447/447 [==============================] - 0s 692us/step - loss: 8.9601e-04 - acc: 1.0000
Epoch 7/150
447/447 [==============================] - 0s 786us/step - loss: 1.5077e-04 - acc: 1.0000
Epoch 8/150
447/447 [==============================] - 0s 699us/step - loss: 1.3581e-04 - acc: 1.0000
Epoch 9/150
447/447 [==============================] - 0s 716us/step - loss: 1.2639e-04 - acc: 1.0000
Epoch 10/150
447/447 [==============================] - 0s 738us/step - loss: 1.1758e-04 - acc: 1.00

In [43]:
_, accuracy = model.evaluate(X, dummy_y)
print('Accuracy: %.2f' % (accuracy*100))

447/447 [==============================] - 0s 774us/step
Accuracy: 100.00


In [46]:
model.save('modelV1')
from keras.models import load_model
loaded_model=load_model('modelV1')
predictions = loaded_model.predict_classes(X)
l = list(map(lambda x:x[0]==x[1],list(zip(predictions,Y))))
sum(l)/len(l)

1.0

In [54]:
df = pd.read_pickle('trainingEmbeddings/DIY015')
print(df['features'][0].shape)
print(df['features'][0])

(10240,)
[0.09120608 0.04207429 0.55686015 ... 0.06631491 0.4375319  0.        ]
